# **`Terrain Demo Project with Perlin Noise`**

<a id="table_of_content"></a>
<a href="../PearlNoiseProject.ipynb">Back To Main</a>
### Table of Contents
1. <a href="#introduction">Introduction</a>
2. <a href="#implementation">Implementation</a>
    - <a href="#perlin_function">Perlin Noise Function</a>
    - <a href="#terrain_generation">Generating Terrain</a>
    - <a href="#terrain_visualization">Visualizing Terrain</a>
    - <a href="#define_water_and_flat">Define Water and Flat</a>
    - <a href="#define_visualize_slope">Define Visualize slope</a>
    - <a href="#plot_height_distribution">Define Plot height distribution</a>
    - <a href="#visualize_water_flat_areas">Define Visualize water flat areas</a>
    - <a href="#visualize_water_area">Define Visualize water area</a>
    - <a href="#visualize_flat_areas">Define Visualize flat areas</a>
    - <a href="#variables_interactive_visualization">Define the Variables for interactive visualization</a>
    - <a href="#interactive_visualization">Define the Interactive visualization</a>
    - <a href="#visualization_layouts">Define the Visualization Layouts</a>
3. <a href="#analysis">Analysis</a>
4. <a href="#conclusions">Conclusion</a>

### Dependencies

First, let's import the required libraries for this project.

In [1]:
import numpy as np
from noise import snoise2, snoise3
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.colors import to_rgba
from mpl_toolkits.mplot3d import Axes3D
import ipywidgets as widgets
from ipywidgets import interact
from matplotlib.colors import ListedColormap
from IPython.display import clear_output, display, HTML

<h2 id="introduction">Introduction</h2>

<a href="#table_of_content">Table of content</a>

In this Demo, we will implement Perlin Noise to generate and visualize terrain using Python and various libraries.

---

<h2 id="implementation">Implementation</h2>

<a href="#table_of_content">Table of content</a>

In this demo, we will implement Perlin Noise to generate and visualize 3D terrain. We will first define several functions to generate and visualize the terrain. Finally, we will create an interactive interface to explore the different parameters of the terrain generation.

<a id="perlin_function"></a>
### Perlin Noise Function
<a href="#table_of_content">Table of content</a>

We will now define the Perlin Noise function to generate terrain. This function will take several parameters, including the size of the terrain, the scale of the noise, the number of octaves, and the persistence and lacunarity values. It will return a 2D array representing the terrain.


In [2]:
def generate_perlin_noise_3d(width, height, depth, scale, height_scale):
    noise = np.zeros((width, height, depth))
    for x in range(width):
        for y in range(height):
            for z in range(depth):
                noise[x, y, z] = snoise3(x * scale, y * scale, z * scale) * height_scale
    return noise

<a id="terrain_generation"></a>
### Generating Terrain
<a href="#table_of_content">Table of content</a>

Now that we have the Perlin Noise function, we can generate the terrain. We will use the function to create a 2D array of noise values, which will represent the terrain's heightmap.

**create_custom_cmap(water_level, terrain_colormap)**: This function creates a custom colormap, where areas that are at or below a certain elevation (defined by `water_level`) are colored in blue (like water), and the rest of the areas use the colormap defined by `terrain_colormap`. The following code demonstrates how to use the Perlin Noise function to generate terrain.


In [3]:
def create_custom_cmap(water_level, terrain_colormap):
    cmap = plt.get_cmap(terrain_colormap)
    colors = cmap(np.arange(cmap.N))
    water_index = int((water_level + 1) / 2 * cmap.N)
    colors[:water_index, :] = np.array([0, 0, 1, 1])  # Задаване на син цвят за водата
    return ListedColormap(colors)


<a id="terrain_visualization"></a>
### Visualizing Terrain
<a href="#table_of_content">Table of content</a>

We will now create a function to visualize the generated terrain using matplotlib. This function will take the terrain array and display it as a 2D plot. It will also allow for customization of the colormap and other visual properties.

**visualize_terrain_with_clouds(noise_3d, water_level, clouds, cloud_threshold, scale, elev, azim, cloud_color, terrain_colormap)**: This function creates a 3D visualization of a terrain represented by a 3D array of Perlin noise (`noise_3d`). The `water_level` parameter defines the cutoff for the terrain to be colored as water. The `clouds` parameter is a 2D array representing the presence and density of clouds. Clouds are displayed where the cloud density exceeds a certain threshold (`cloud_threshold`). The `scale`, `elev`, and `azim` parameters define the scale and viewing angle of the 3D plot. The `cloud_color` parameter is the color of the clouds, and `terrain_colormap` is the colormap of the terrain.

In [4]:
def visualize_terrain_with_clouds(noise_3d, water_level, clouds, cloud_threshold, scale, elev, azim, cloud_color, terrain_colormap):
    fig = plt.figure(figsize=(12, 12))
    ax = fig.add_subplot(111, projection='3d')
    X, Y = np.meshgrid(range(noise_3d.shape[0]), range(noise_3d.shape[1]), indexing='ij')
    Z = np.mean(noise_3d, axis=2)
    custom_cmap = create_custom_cmap(water_level, terrain_colormap)
    ax.plot_surface(X, Y, Z, cmap=custom_cmap)
        
    cloud_mask = clouds > cloud_threshold
    cloud_x, cloud_y = np.where(cloud_mask)
    cloud_z = (Z.max() + 5) * (1 + cloud_x * 0)
    cloud_colors = np.zeros((cloud_x.size, 4))
    cloud_colors[:, :4] = to_rgba(cloud_color)
    cloud_colors[:, 3] = (clouds[cloud_mask] - cloud_threshold) / (1 - cloud_threshold)
    ax.scatter(cloud_x, cloud_y, cloud_z, c=cloud_colors, s=10)
    
    ax.view_init(elev=elev, azim=azim)
    
    plt.show()


<a id="define_water_and_flat"></a>
### Define the calculate_water_and_flat_areas function
<a href="#table_of_content">Table of content</a>

**calculate_water_and_flat_areas(heights, water_level, flatness_threshold)**: This function calculates the percentage of a terrain that is covered by water and that is flat. The `heights` parameter is a 2D array representing the terrain's heights. Areas below the `water_level` are considered water. Areas with a slope less than `flatness_threshold` are considered flat.

In [5]:
def calculate_water_and_flat_areas(heights, water_level, flatness_threshold):
    water_percentage = (np.sum(heights <= water_level) / heights.size) * 100
    
    dy, dx = np.gradient(heights)
    slope = np.sqrt(dy**2 + dx**2)
    flat_mask = slope < flatness_threshold
    flat_percentage = (np.sum(flat_mask) / heights.size) * 100
    
    return water_percentage, flat_percentage

<a name="define_water_and_flat"></a>
### Define the generate_clouds function
<a href="#table_of_content">Table of content</a>

**generate_clouds(width, height, scale)**: This function generates a 2D array of cloud densities using Perlin noise. The `width` and `height` parameters define the size of the output array, and `scale` defines the frequency of the Perlin noise.

In [6]:
def generate_clouds(width, height, scale):
    clouds = np.zeros((width, height))
    for x in range(width):
        for y in range(height):
            clouds[x, y] = snoise3(x * scale, y * scale, 0)
            
    return clouds

<a id="define_visualize_slope"></a>
### Define the visualize_slope function
<a href="#table_of_content">Table of content</a>

**visualize_slope(heights, cmap)**: This function visualizes the slope of a terrain. The `heights` parameter is a 2D array representing the terrain's heights, and `cmap` is the colormap for the visualization.

In [7]:
def visualize_slope(heights, cmap):
    dy, dx = np.gradient(heights)
    slope = np.sqrt(dy**2 + dx**2)
    fig, ax = plt.subplots()
    im = ax.imshow(slope, cmap=cmap, origin='lower')
    fig.colorbar(im, ax=ax, label='Slope')
    plt.show()


<a id="plot_height_distribution"></a>
### Define the plot_height_distribution function
<a href="#table_of_content">Table of content</a>

**plot_height_distribution(heights)**: This function creates a histogram of the heights of a terrain, effectively showing the height distribution. The `heights` parameter is a 2D array representing the terrain's heights.

In [8]:
def plot_height_distribution(heights):
    plt.hist(heights.flatten(), bins=50, density=True, alpha=0.75)
    plt.xlabel('Height')
    plt.ylabel('Density')
    plt.show()

<a id="visualize_water_flat_areas"></a>
### Define the visualize_water_flat_areas function
<a href="#table_of_content">Table of content</a>

**visualize_water_flat_areas(heights, water_level, flatness_threshold)**: This function visualizes water and flat areas in a terrain. The `heights` parameter is a 2D array representing the terrain's heights. Areas below `water_level` are considered water, and areas with a slope less than `flatness_threshold` are considered flat.

In [9]:
def visualize_water_flat_areas(heights, water_level, flatness_threshold):
    water_mask = heights <= water_level
    
    dy, dx = np.gradient(heights)
    slope = np.sqrt(dy**2 + dx**2)
    flat_mask = slope < flatness_threshold
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))
    
    ax1.imshow(water_mask, cmap='Blues_r', origin='lower')
    ax1.set_title('Water Areas')
    
    ax2.imshow(flat_mask, cmap='Greens_r', origin='lower')
    ax2.set_title('Flat Areas')
    
    plt.show()

<a id="visualize_water_area"></a>
### Define the visualize_water_area function
<a href="#table_of_content">Table of content</a>

**visualize_water_area(heights, water_level)**: This function visualizes water areas in a terrain. The `heights` parameter is a 2D array representing the terrain's heights. Areas below `water_level` are considered water.

In [10]:
def visualize_water_area(heights, water_level):
    water_mask = heights <= water_level
    
    fig, ax = plt.subplots(figsize=(6, 6))
    
    from matplotlib.colors import ListedColormap
    colors = ["sienna", "blue"]
    custom_cmap = ListedColormap(colors)
    
    ax.imshow(water_mask, cmap=custom_cmap, origin='lower')
    ax.set_title('Water Area')
    
    plt.show()

<a id="visualize_flat_areas"></a>
### Define the visualize_flat_areas function
<a href="#table_of_content">Table of content</a>

**visualize_flat_areas(heights, flatness_threshold)**: This function visualizes flat areas in a terrain. The `heights` parameter is a 2D array representing the terrain's heights. Areas with a slope less than `flatness_threshold` are considered flat.

In [11]:
def visualize_flat_areas(heights, flatness_threshold):
    dy, dx = np.gradient(heights)
    slope = np.sqrt(dy**2 + dx**2)
    flat_mask = slope < flatness_threshold
    
    fig, ax = plt.subplots(figsize=(6, 6))
    
    ax.imshow(flat_mask, cmap='Greens_r', origin='lower')
    ax.set_title('Flat Areas')
    
    plt.show()

<a id="variables_interactive_visualization"></a>
### Defining the variables for interactive_visualization
<a href="#table_of_content">Table of content</a>

In [12]:
slope_output = widgets.Output()
height_distribution_output = widgets.Output()
water_area_output = widgets.Output()
flat_areas_output = widgets.Output()

<a id="interactive_visualization"></a>
### Define the interactive_visualization function
<a href="#table_of_content">Table of content</a>

The provided code defines an interactive 3D visualization of a generated terrain, using the concept of Perlin noise. The terrain can be customized by adjusting various parameters such as the size of the terrain (width, height, depth), scale, water level, flatness threshold, cloud parameters, viewpoint parameters, cloud color, height scale, and color map of the terrain.

The final application is composed of several parts, including a main 3D visualization of the terrain with clouds, and smaller visualizations of the terrain's slope, height distribution, water areas, and flat areas. Let's dive into the details:

**Interactive Visualization Function**: This function (`interactive_visualization`) performs the following tasks:
   - It first generates a 3D Perlin noise which simulates a terrain and creates a cloud cover.
   - It visualizes the 3D terrain with the clouds using the function `visualize_terrain_with_clouds`.
   - Then, it calculates the average height of the terrain and gets the terrain's color map.
   - It also performs visualizations of slope, height distribution, water area, and flat areas, each outputting to a dedicated widget.

In [13]:
def interactive_visualization(width, height, depth, scale, water_level, flatness_threshold, cloud_scale, cloud_threshold, elev, azim, cloud_color, height_scale, terrain_colormap):
    noise_3d = generate_perlin_noise_3d(width, height, depth, scale, height_scale)
    clouds = generate_clouds(width, height, cloud_scale)
    visualize_terrain_with_clouds(noise_3d, water_level, clouds, cloud_threshold, scale, elev, azim, cloud_color, terrain_colormap)
    
    heights = np.mean(noise_3d, axis=2)
    cmap = plt.get_cmap(terrain_colormap)

    with slope_output:
        clear_output(wait=True)
        visualize_slope(heights, cmap)

    with height_distribution_output:
        clear_output(wait=True)
        plot_height_distribution(heights)

    with water_area_output:
        clear_output(wait=True)
        visualize_water_area(heights, water_level)

    with flat_areas_output:
        clear_output(wait=True)
        visualize_flat_areas(heights, flatness_threshold)    


<a id="visualization_layouts"></a>
### Visualize the 3D terrain
<a href="#table_of_content">Table of content</a>

**Widget Definitions**: This part of the code is defining interactive widgets for user inputs. Widgets are provided for adjusting the terrain's size, scale, water level, flatness threshold, cloud scale, cloud threshold, viewpoint elevation and azimuth, cloud color, height scale, and the color map of the terrain.

**Interactive Output Definition**: This part of the code (`widgets.interactive_output`) connects the defined widgets to the interactive visualization function. As a result, whenever a widget's value is changed, the visualization function is called with the new values, updating the visualization.

**Visualization Layouts**: This part of the code structures the final layout of the application. The `AppLayout` widget is used to arrange the main 3D visualization and the smaller visualizations in a specific layout. The layout includes the main 3D visualization in the center, and the smaller visualizations are displayed to the right of it. The controls for adjusting the parameters are positioned in the left sidebar.

In [14]:
cloud_color_picker = widgets.ColorPicker(
    concise=False,
    description='Cloud color:',
    value='blue',
    disabled=False
)

width_slider = widgets.IntSlider(value=100, min=10, max=500, step=10, description='Width:')
height_slider = widgets.IntSlider(value=100, min=10, max=500, step=10, description='Height:')
depth_slider = widgets.IntSlider(value=100, min=10, max=500, step=10, description='Depth:')
scale_slider = widgets.FloatSlider(value=0.03, min=-0.00, max=0.1, step=0.0005, description='Scale:')
water_level_slider = widgets.FloatSlider(value=-0.50, min=-1, max=1, step=0.01, description='Water level:')
flatness_threshold_slider = widgets.FloatSlider(value=0.01, min=-0.01, max=0.5, step=0.01, description='Flatness:')
cloud_scale_slider = widgets.FloatSlider(value=0.03, min=0.00, max=0.1, step=0.01, description='Cloud scale:')
cloud_threshold_slider = widgets.FloatSlider(value=0.0, min=-1.0, max=1.0, step=0.01, description='Cloud threshold:')
elev_slider = widgets.FloatSlider(value=30, min=-90, max=90, step=1, description='Elevation:')
azim_slider = widgets.FloatSlider(value=-135, min=-180, max=180, step=1, description='Azimuth:')
cloud_color_picker = widgets.ColorPicker(value='lightblue', description='Cloud color:')
height_scale_slider = widgets.FloatSlider(value=12, min=1, max=50, step=1, description='Height Scale:')
terrain_colormap_dropdown = widgets.Dropdown(
    options=plt.colormaps(),
    value="terrain",
    description="Terrain map:"
)

out = widgets.interactive_output(
    interactive_visualization, 
    {
        'width': width_slider, 
        'height': height_slider, 
        'depth': depth_slider, 
        'scale': scale_slider, 
        'water_level': water_level_slider, 
        'flatness_threshold': flatness_threshold_slider, 
        'cloud_scale': cloud_scale_slider, 
        'cloud_threshold': cloud_threshold_slider, 
        'elev': elev_slider, 
        'azim': azim_slider, 
        'cloud_color': cloud_color_picker,
        'height_scale': height_scale_slider,
        'terrain_colormap': terrain_colormap_dropdown
    }
)

controls = widgets.VBox(
    [
        width_slider,
        height_slider,
        depth_slider,
        scale_slider,
        water_level_slider,
        flatness_threshold_slider,
        cloud_scale_slider,
        cloud_threshold_slider,
        elev_slider,
        azim_slider,
        cloud_color_picker,
        height_scale_slider,
        terrain_colormap_dropdown
    ]
)


for control in controls.children:
    control.layout.width = '300px'

header_html = widgets.HTML('<h1>Interactive 3D Visualization</h1>')
display(header_html)

out.layout.width = '450px'
out.layout.height = '500px'
other_visualization_width = '200px'
other_visualization_height = '220px'
slope_output.layout.width = other_visualization_width
slope_output.layout.height = other_visualization_height
height_distribution_output.layout.width = other_visualization_width
height_distribution_output.layout.height = other_visualization_height
water_area_output.layout.width = other_visualization_width
water_area_output.layout.height = other_visualization_height
flat_areas_output.layout.width = other_visualization_width
flat_areas_output.layout.height = other_visualization_height

app = widgets.AppLayout(
    header=None,
    left_sidebar=controls,
    center=widgets.HBox([
        widgets.VBox([out], layout=widgets.Layout(padding='0px')),
        widgets.VBox([
            widgets.HBox([height_distribution_output, slope_output], layout=widgets.Layout(padding='0px')),
            widgets.HBox([water_area_output, flat_areas_output], layout=widgets.Layout(padding='0px'))
        ], layout=widgets.Layout(padding='0px')),
    ], layout=widgets.Layout(align_items='center', padding='0px', width='70%')),
    right_sidebar=None,
    footer=None,
    layout=widgets.Layout(width='105%', align_items='center', display='flex', padding='0px')
)

display(app)

HTML(value='<h1>Interactive 3D Visualization</h1>')

AppLayout(children=(VBox(children=(IntSlider(value=100, description='Width:', layout=Layout(width='300px'), ma…

<a id="analysis"></a>

## Analysis

<a href="#table_of_content">Table of content</a>

<a href="../PearlNoiseProject.ipynb">Back To Main</a>

In this section, we will analyze the generated terrain and discuss the impact of different parameters on the final output. The Perlin Noise function allows for the adjustment of several parameters, including the number of octaves, persistence, and lacunarity. Changing these values can have a significant impact on the appearance and structure of the generated terrain. By experimenting with different values, we can create a wide range of terrains suitable for various applications.

The code above defines an application for generating and visualizing 3D terrains. It uses Perlin noise to create realistic heightmaps, which are then plotted. Several additional visualizations, such as slope and height distribution, are provided alongside the main 3D view.

**Application Analysis**

This application demonstrates a successful integration of data generation and interactive visualization.

1. **Realistic Terrain Generation**: Perlin noise is a gradient noise developed by Ken Perlin, often used in procedural texture and terrain generation for its organic and smooth appearance. This application uses a 3D implementation of Perlin noise, which allows for an added depth dimension, enhancing the terrain's realism.

2. **Interactive Visualization**: The application includes an interactive 3D visualization of the generated terrain, complete with a customizable water level and cloud cover. This provides an intuitive and engaging way to explore and understand the terrain.

3. **Additional Visualizations**: Besides the main 3D plot, the application includes several other visualizations providing different perspectives on the terrain's properties. These include plots of slope, height distribution, water areas, and flat areas.

4. **User-Friendly Interface**: The interactive widgets enables to modify various parameters affecting the terrain and its visualization easily. These widgets include sliders for numerical inputs, color pickers, and dropdown menus.

**Conclusions**

In conclusion, this application is a powerful tool for generating and visualizing 3D terrains. It is not only technically robust but also user-friendly and visually engaging. Potential improvements could include adding more parameters for the user to modify or incorporating other types of noise generation.

Possible applications of this tool could be in geography and geology education, video game design, or any area where realistic terrain visualization could be beneficial. It serves as an excellent example of how Python and Jupyter's interactive capabilities can be leveraged to create effective and interactive scientific visualizations.


<a id="conclusions"></a>

## Conclusions

<a href="#table_of_content">Table of content</a>

<a href="../PearlNoiseProject.ipynb">Back To Main</a>

In this demo, we created and analyzed the generated terrain and discuss the impact of different parameters on the final output. The Perlin Noise function allows for the adjustment of several parameters, including the number of octaves, persistence, and lacunarity. Changing these values can have a significant impact on the appearance and structure of the generated terrain. By experimenting with different values, we can create a wide range of terrains suitable for various applications.

In conclusion, this application is a tool for generating and visualizing 3D terrains. It is not only technically robust but also user-friendly and visually engaging. Potential improvements could include adding more parameters to modify or incorporating other types of noise generation.

Possible applications of this tool could be in geography and geology education, video game design, or any area where realistic terrain visualization could be beneficial. It serves as an excellent example how to create effective and interactive scientific visualizations.
